In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms, models
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
import numpy as np

# Configuration
BATCH_SIZE = 32 * 2  # Scaled for 2 GPUs
LEARNING_RATE = 0.001
EPOCHS = 10
IMG_SIZE = 224
DATA_DIR = '/kaggle/input/augmented-alzheimer-mri-dataset/AugmentedAlzheimerDataset'

# Device Configuration for Multi-GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
else:
    print("Warning: 2 GPUs not found. Training will proceed on available hardware.")

In [ ]:
def safe_loader(path):
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    except (UnidentifiedImageError, OSError):
        print(f"Skipping corrupt image: {path}")
        # Return a black image to prevent crashing, or handle explicitly in dataset
        return Image.new('RGB', (IMG_SIZE, IMG_SIZE))

# Define Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Load Dataset with specific loader
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform, loader=safe_loader)

# Split into Train and Validation (80/20 split)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print(f"Classes: {full_dataset.classes}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

In [ ]:
def get_model(num_classes):
    model = models.resnet50(pretrained=True)
    
    # Freeze early layers (optional, usually good for small med datasets)
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace the final fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    
    return model

model = get_model(len(full_dataset.classes))

# Apply DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-" * 10)

        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # --- Validation Phase ---
        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")
        print()

    return train_losses, train_accs, val_losses, val_accs

# Start Training
history = train_model(model, train_loader, val_loader, criterion, optimizer, EPOCHS)

In [ ]:
print("Unfreezing model for fine-tuning...")

# 1. Unfreeze all layers
for param in model.parameters():
    param.requires_grad = True

# 2. Lower the learning rate significantly
# We lower it to 1e-4 or 1e-5 to avoid destroying the pre-trained weights
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# 3. Train for more epochs (Fine-tuning usually takes longer)
FINE_TUNE_EPOCHS = 10

# Re-use the same training function
history_ft = train_model(model, train_loader, val_loader, criterion, optimizer, FINE_TUNE_EPOCHS)

# 4. Combine histories for plotting
train_losses += history_ft[0]
train_accs += history_ft[1]
val_losses += history_ft[2]
val_accs += history_ft[3]

# Plot updated results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.axvline(x=EPOCHS, color='r', linestyle='--', label='Unfreeze Point') # Mark where we unfroze
plt.legend()
plt.title('Loss (Feature Extract + Fine Tune)')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(val_accs, label='Val Acc')
plt.axvline(x=EPOCHS, color='r', linestyle='--', label='Unfreeze Point')
plt.legend()
plt.title('Accuracy (Feature Extract + Fine Tune)')
plt.show()

In [ ]:
# Plotting results
train_losses, train_accs, val_losses, val_accs = history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss per Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(val_accs, label='Val Acc')
plt.title('Accuracy per Epoch')
plt.legend()

plt.show()

In [ ]:
# Save the trained model weights
torch.save(model.state_dict(), 'alzheimer_resnet50.pth')
print("Model saved successfully as 'alzheimer_resnet50.pth'")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd

def evaluate_model(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    
    # Disable gradient calculation for inference
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            # Move to CPU to use with sklearn
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            
    return y_true, y_pred

# Get predictions
print("Generating evaluation metrics...")
y_true, y_pred = evaluate_model(model, val_loader)

# 1. Classification Report
class_names = full_dataset.classes
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# 2. Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def predict_image(image_path, model):
    # Load and Preprocess
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0) # Add batch dimension
    
    # Predict
    model.eval()
    with torch.no_grad():
        img_tensor = img_tensor.to(device)
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        
    class_name = full_dataset.classes[predicted.item()]
    
    plt.imshow(img)
    plt.title(f"Prediction: {class_name}")
    plt.axis('off')
    plt.show()

# Example usage (replace with a real path from the dataset)
sample_path = "/kaggle/input/augmented-alzheimer-mri-dataset/OriginalDataset/ModerateDemented/28.jpg"
predict_image(sample_path, model)